In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

import time
import numpy as np
import pathlib
import time
import datetime 
import json
import random
import sys 
import os
# CHANGE THIS TO NAME OF CURRENT NOTEBOOK!
CURR_NOTEBOOK = "twoblstm32"

print("Last run time: ", datetime.datetime.now())
print("Files in Directory: " ,os.listdir("drive/My Drive/Colab Notebooks"))
print("Absolute Path:", pathlib.Path().absolute())
print("Current Notebook: ", CURR_NOTEBOOK)

path = "drive/My Drive/Colab Notebooks/"

# Make Models data directory if it doesn't exist
if not os.path.exists(path + "models/" + CURR_NOTEBOOK):
    os.makedirs(path + "models/"+ CURR_NOTEBOOK)
    print(CURR_NOTEBOOK  + " directory created at " + path + CURR_NOTEBOOK)

# Check CUDA
cuda = torch.cuda.is_available()

# Seed Random Number generator for CPU & CUDA
torch.manual_seed(1)
if cuda:
    torch.cuda.manual_seed(1)
kwargs = {'num_workers': 2, 'pin_memory': True} if cuda else {}


Last run time:  2020-04-24 01:04:54.998015
Files in Directory:  ['data', 'models', 'PreprocessData-2.ipynb', 'GRU-Rand.ipynb', 'Old', 'TransferLSTM.ipynb', 'AppendData.ipynb', 'cs7643colab.ipynb', 'hyperparams.json', 'Ntb_Att-GRU.ipynb', 'TestLSTM.ipynb', 'Ntb_GRU.ipynb', 'SingleLSTM32.ipynb', 'TwoBLSTMs.ipynb', 'TestLSTM32.ipynb', 'Model Notes.gdoc', 'LSTM-GRU-Rand.ipynb', 'NTB_GRU-LSTM32.ipynb', 'meetingnotes.gdoc', 'NTB_GRU32.ipynb', 'TestBERT.ipynb', 'TestBERT32.ipynb', 'TwoBLSTM32.ipynb']
Absolute Path: /content
Current Notebook:  twoblstm32


In [6]:
MAX_SEQ_LEN = 32
FRAC_OF_TRAIN = .70
FRAC_OF_VAL = .20
FRAC_OF_TEST = .10

### Assign path to master file of data
words_file_master = path + "data/32/TED_words_master_32_q.txt"
punct_file_master = path + "data/32/TED_punc_master_32_q.txt"

print(words_file_master)

drive/My Drive/Colab Notebooks/data/32/TED_words_master_32_q.txt


In [7]:
class PunctuationDataset(Dataset):
    def __init__(self, split='train'):
        self.split = split
        self.train_data = []
        self.train_labels = []
        self.val_data = []
        self.val_labels = []
        self.test_data = []
        self.test_labels = []
        self.word_to_ix = {}
        self.tag_to_ix = {'_': 0, '.': 1, ',': 2, '?' : 3}
        
        # w_file = open(words_file, 'r',encoding='utf-8') 
        # p_file = open(punct_file, 'r',encoding='utf-8')

        input_file = open (path + "models/indices32.json")
        json_array = json.load(input_file)
        train_idx = json_array["train_idx"]
        val_idx = json_array["val_idx"]
        test_idx = json_array["test_idx"]

        #####################################################################################
        #                                IN PROGRESS                                        #
        #####################################################################################
        # with open(words_file_master) as f:
        #   words_master_total_lines = sum(1 for _ in f)

        # with open(punct_file_master) as f:
        #   punc_master_total_lines = sum(1 for _ in f)

        # print("Total Number of Lines: ", words_master_total_lines, \
        #       "Total Number of Punc Lines: ", punc_master_total_lines)
        
        # # Get Size of training sets (set fractions above)
        # SIZE_OF_TRAIN_WORDS = int(np.floor(FRAC_OF_TRAIN * words_master_total_lines))
        # SIZE_OF_VAL_WORDS = int(np.ceil(FRAC_OF_VAL * words_master_total_lines))
        # SIZE_OF_TEST_WORDS =  int(words_master_total_lines - SIZE_OF_TRAIN_WORDS - SIZE_OF_VAL_WORDS)
 
        # print("Training set size: ", SIZE_OF_TRAIN_WORDS, ", Validation Set Size: ", SIZE_OF_VAL_WORDS, \
        #       ", Test Set Size: ", SIZE_OF_TEST_WORDS)
        
        # # Get randomly generated indices
        # train_idx = random.sample(range(0, words_master_total_lines), SIZE_OF_TRAIN_WORDS) # Get random sample for training
        # remaining_idx = [idx for idx in range(0, words_master_total_lines) if idx not in train_idx] # Get indices not in training set
        # val_idx = random.sample(remaining_idx, SIZE_OF_VAL_WORDS) # Get random subset of remaining indices
        # test_idx = [idx for idx in remaining_idx if idx not in val_idx] # Get remaining indices for test set
        # save_indices(train_idx, val_idx, test_idx)

        # Read the lines in the master file
        wordslst = open(words_file_master).readlines()
        punclst  = open(punct_file_master).readlines()

        # Get assign sets from files
        words_training_set = [wordslst[i] for i in train_idx]
        words_val_set = [wordslst[i] for i in val_idx]
        words_test_set = [wordslst[i] for i in test_idx]

        punc_training_set = [punclst[i] for i in train_idx]
        punc_val_set = [punclst[i] for i in val_idx]
        punc_test_set = [punclst[i] for i in test_idx]
        # print(templst[int(test_idx)])
        #####################################################################################
        #                                IN PROGRESS                                        #
        #####################################################################################

        # TRAINING DATA
        # 
        # Loop through lines in word & punctuation file
        # for la, lb in zip(w_file, p_file):
        for la, lb in zip(words_training_set, punc_training_set):
            words = la.split()
            punc_lst = list(lb[:-1])

            if len(words) != len(punc_lst) or len(words) != MAX_SEQ_LEN: #TODO: need to fix preprocessing
                continue

           # Loop through words & save length of word
            for i in range(len(words)):
                w = words[i]
                
                # Save word if not in word_to_ix w/ respective index
                if w not in self.word_to_ix:
                    self.word_to_ix[w] = len(self.word_to_ix)
                words[i] = self.word_to_ix[w]
            
            # Loop through punctuation & assign proper index tag
            for i in range(len(punc_lst)):
                punc_lst[i] = self.tag_to_ix[punc_lst[i]]

            self.train_data.append(words)
            self.train_labels.append(punc_lst)
            
        # w_file.close()
        # p_file.close()

        self.train_data = torch.from_numpy(np.matrix(self.train_data, dtype='int64'))
        self.train_labels = torch.from_numpy(np.matrix(self.train_labels, dtype='int64'))
        
        #TODO: use val data
#         self.val_data = self.train_data
#         self.val_labels = self.train_labels
        # w_file = open(words_file_dev, 'r',encoding='utf-8') 
        # p_file = open(punct_file_dev, 'r',encoding='utf-8')
        
        # TRAINING DATA
        # 
        # Loop through lines in word & punctuation file
        # for la, lb in zip(w_file, p_file):
        for la, lb in zip(words_val_set, punc_val_set):
            words = la.split()
            punc_lst = list(lb[:-1])

            if len(words) != len(punc_lst) or len(words) != MAX_SEQ_LEN: #TODO: need to fix preprocessing
                continue

            for i in range(len(words)):
                w = words[i]
                if w not in self.word_to_ix:
                    self.word_to_ix[w] = len(self.word_to_ix)
                words[i] = self.word_to_ix[w]
            
            for i in range(len(punc_lst)):
                punc_lst[i] = self.tag_to_ix[punc_lst[i]]
            
            self.val_data.append(words)
            self.val_labels.append(punc_lst)
            
        # w_file.close()
        # p_file.close()
        self.val_data = torch.from_numpy(np.matrix(self.val_data, dtype='int64'))
        self.val_labels = torch.from_numpy(np.matrix(self.val_labels, dtype='int64'))
        
        #TODO: use test data
        for la, lb in zip(words_test_set, punc_test_set):
            words = la.split()
            punc_lst = list(lb[:-1])

            if len(words) != len(punc_lst) or len(words) != MAX_SEQ_LEN: #TODO: need to fix preprocessing
                continue

            for i in range(len(words)):
                w = words[i]
                if w not in self.word_to_ix:
                    self.word_to_ix[w] = len(self.word_to_ix)
                words[i] = self.word_to_ix[w]
            
            for i in range(len(punc_lst)):
                punc_lst[i] = self.tag_to_ix[punc_lst[i]]
            
            self.test_data.append(words)
            self.test_labels.append(punc_lst)
            
        # w_file.close()
        # p_file.close()
        self.test_data = torch.from_numpy(np.matrix(self.test_data, dtype='int64'))
        self.test_labels = torch.from_numpy(np.matrix(self.test_labels, dtype='int64'))
 
    def __len__(self):
        if self.split == 'train':
            return len(self.train_data)
        elif self.split == 'val':
            return len(self.val_data)
        else:
            return len(self.test_data)
 
    def __getitem__(self,idx):
        if self.split == 'train':
            return (self.train_data[idx], self.train_labels[idx])
        elif self.split == 'val':
            return (self.val_data[idx], self.val_labels[idx])
        else:
            return (self.test_data[idx], self.test_labels[idx])
        
    def set_split(self, split):
        self.split = split
 
punc_dataset = PunctuationDataset()
print("Size of Training Data: ", len(punc_dataset.train_data))
print("Size of Validation Data: ", len(punc_dataset.val_data))
print("Size of Test Data: ", len(punc_dataset.test_data))

Size of Training Data:  49476
Size of Validation Data:  14136
Size of Test Data:  7064


In [8]:
# Hyperparameters
# EMBEDDING_DIM = 64
# HIDDEN_DIM = 256
# LEARNING_RATE = 0.1
# WEIGHT_DECAY = 0
# EPOCHS = 5
# MOMENTUM = 0
input_file = open (path + "/hyperparams.json")
json_array = json.load(input_file)
JSON_IDX = 1
params = json_array[JSON_IDX]

EMBEDDING_DIM = params["EMBEDDING_DIM"]
HIDDEN_DIM = params["HIDDEN_DIM"]
LEARNING_RATE = params["LEARNING_RATE"]
WEIGHT_DECAY = params["WEIGHT_DECAY"]
EPOCHS = params["EPOCHS"]
MOMENTUM = params["MOMENTUM"]
BATCH_SIZE = params["BATCH_SIZE"]

#HArd coding for now
BATCH_SIZE = 16
LEARNING_RATE = 1e-4
EMBEDDING_DIM = 1024

print(params)

{'EMBEDDING_DIM': 64, 'HIDDEN_DIM': 256, 'LEARNING_RATE': 0.1, 'WEIGHT_DECAY': 0.0, 'EPOCHS': 100, 'MOMENTUM': 0.9, 'BATCH_SIZE': 32}


In [0]:
train_loader = DataLoader(punc_dataset,BATCH_SIZE, kwargs)

In [0]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, tagset_size, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim*2, hidden_dim, tagset_size, bidirectional=True, batch_first=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
#         print(embeds.shape)
#         lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        lstm_1,_ = self.lstm(embeds)
        lstm_out,_ = self.lstm2(lstm_1)
#         print(lstm_out.shape)
#         tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_space = self.hidden2tag(lstm_out)
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [0]:
def save_model(epoch):
    path = "drive/My Drive/Colab Notebooks/models/" + CURR_NOTEBOOK + "/"
    model_name = "model_" + str(JSON_IDX) + "_" + time.strftime("%m%d%Y_%H%M") + "_" + str(epoch)
    torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': running_loss,
                }, path + model_name)
    print(model_name, "saved.")
    return model_name

In [0]:
def get_val_accuracy(verbose=False):
    # See what the scores are after training
    scores = []
    true_pos = {0:0, 1:0, 2:0, 3:0}
    false_neg = {0:0,1:0,2:0, 3:0}
    false_pos = {0:0,1:0,2:0,3:0}
    step = 1000   # How many instances to process at once. Should fix memory issue
    with torch.no_grad():
        for k in range(0, len(punc_dataset.val_data)-step, step):
            inputs = punc_dataset.val_data[k:k+step]
            outputs = punc_dataset.val_labels[k:k+step]
            if cuda:
                inputs = inputs.cuda()
                outputs = outputs.cuda()

            tag_scores = model(inputs)
            scores = torch.argmax(tag_scores, axis=2).cpu().numpy()


            for i in range(len(scores)):
                for j in range(len(scores[i])):
                    op = outputs[i][j].item()
                    if op == scores[i][j]:
                        true_pos[op] += 1
                    else:
                        false_pos[scores[i][j]] += 1
                        false_neg[op] += 1
    if verbose:
        print(true_pos)
        print(false_pos)
        print(false_neg)
    
    for i in range(1, len(punc_dataset.tag_to_ix)):
        pre = true_pos[i] / (true_pos[i] + false_pos[i])
#         print("Precision of ",i,": ", pre)
        rec = true_pos[i] / (true_pos[i] + false_neg[i])
#         print("Recall of " , i , ": ", rec)
        f_score = 2 * pre * rec / (pre + rec)
#         print("F score of " , i , ": " , f_score)
        if verbose:
            print("Precision, Recall, F score of", i, ": ", pre, rec, f_score)
        
    total_true_pos = 0
    total_false_pos = 0
    total_false_neg = 0

    for i in range(1, len(punc_dataset.tag_to_ix)):
        total_true_pos += true_pos[i]
        total_false_pos += false_pos[i]
        total_false_neg += false_neg[i]

    pre = total_true_pos / (total_true_pos + total_false_pos)
#     print("Total precision: ", pre)
    rec = total_true_pos / (total_true_pos + total_false_neg)
#     print("Total recall: ", rec)
    f_score = 2 * pre * rec / (pre + rec)
#     print("Total F1 score ", f_score)
    print("Total Precision, Recall, F score: ", pre, rec, f_score)
    return f_score

In [0]:
def get_test_accuracy(verbose=False):
    # See what the scores are after training
    scores = []
    true_pos = {0:0, 1:0, 2:0, 3:0}
    false_neg = {0:0,1:0,2:0, 3:0}
    false_pos = {0:0,1:0,2:0,3:0}
    step = 1000   # How many instances to process at once. Should fix memory issue
    with torch.no_grad():
        for k in range(0, len(punc_dataset.test_data)-step, step):
            inputs = punc_dataset.test_data[k:k+step]
            outputs = punc_dataset.test_labels[k:k+step]
            if cuda:
                inputs = inputs.cuda()
                outputs = outputs.cuda()

            tag_scores = model(inputs)
            scores = torch.argmax(tag_scores, axis=2).cpu().numpy()


            for i in range(len(scores)):
                for j in range(len(scores[i])):
                    op = outputs[i][j].item()
                    if op == scores[i][j]:
                        true_pos[op] += 1
                    else:
                        false_pos[scores[i][j]] += 1
                        false_neg[op] += 1
    if verbose:
        print(true_pos)
        print(false_pos)
        print(false_neg)
    
    for i in range(1, len(punc_dataset.tag_to_ix)):
        pre = true_pos[i] / (true_pos[i] + false_pos[i])
#         print("Precision of ",i,": ", pre)
        rec = true_pos[i] / (true_pos[i] + false_neg[i])
#         print("Recall of " , i , ": ", rec)
        f_score = 2 * pre * rec / (pre + rec)
#         print("F score of " , i , ": " , f_score)
        if verbose:
            print("TEST Precision, Recall, F score of", i, ": ", pre, rec, f_score)
        
    total_true_pos = 0
    total_false_pos = 0
    total_false_neg = 0

    for i in range(1, len(punc_dataset.tag_to_ix)):
        total_true_pos += true_pos[i]
        total_false_pos += false_pos[i]
        total_false_neg += false_neg[i]

    pre = total_true_pos / (total_true_pos + total_false_pos)
#     print("Total precision: ", pre)
    rec = total_true_pos / (total_true_pos + total_false_neg)
#     print("Total recall: ", rec)
    f_score = 2 * pre * rec / (pre + rec)
#     print("Total F1 score ", f_score)
    print("TEST Total Precision, Recall, F score: ", pre, rec, f_score)
    return f_score

In [14]:
load = True

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(punc_dataset.word_to_ix), len(punc_dataset.tag_to_ix))
if cuda:
  model.cuda()
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY) #, momentum=MOMENTUM)

start_epoch = 1

if load:
  #LOAD_MODEL
  model_name = "model_1_04232020_1705_50"
  # Hard coding learning rate decay
  # WEIGHT_DECAY = 0.99
  model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(punc_dataset.word_to_ix), len(punc_dataset.tag_to_ix))
  optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY) # momentum=MOMENTUM)
  if cuda:
    model.cuda()
  path = "drive/My Drive/Colab Notebooks"
  checkpoint = torch.load(path + "/models/" + CURR_NOTEBOOK + "/" + model_name)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  start_epoch = checkpoint['epoch']
  loss = checkpoint['loss']
  print("Epoch from loaded model: ", start_epoch)
  print("Loss from loaded model: ", loss.item())



Epoch from loaded model:  50
Loss from loaded model:  9264.298828125


In [0]:
running_loss = 0.0
prev_f1 = 0.0
SAVE_FREQ = 10
print("Start Training! ")
start_time = time.time()
for epoch in range(start_epoch, EPOCHS + start_epoch + 1):
    running_loss = 0.0
    for batch_idx, batch in enumerate(train_loader):
        model.zero_grad()
        sentence_in, targets = Variable(batch[0]), Variable(batch[1])
        targets = targets.view(-1)
        
        if cuda:
            sentence_in = sentence_in.cuda()
            targets = targets.cuda()
            
        tag_scores = model(sentence_in)
        tag_scores = tag_scores.view(-1, 4)

        loss = loss_function(tag_scores, targets)
        running_loss += loss
        loss.backward()
        optimizer.step()
    print("Epoch: ", epoch, "Loss: ", running_loss.item())
    # print("F1 Score:", get_val_accuracy())
    if epoch % SAVE_FREQ == 0:
        curr_f1 = get_val_accuracy()
        print("F1 Score", curr_f1)
        if curr_f1 > prev_f1:
            prev_f1 = curr_f1
            save_model(epoch)
            
print("Run time: ",(time.time() - start_time)/60, "minutes")        
get_val_accuracy(True)
model_name = save_model(EPOCHS+start_epoch)



Start Training! 
Epoch:  30 Loss:  9271.6875
Total Precision, Recall, F score:  0.49718050604803365 0.5505443486419593 0.5225034351994718
F1 Score 0.5225034351994718
model_1_04232020_1500_30 saved.
Epoch:  31 Loss:  9270.6904296875
Epoch:  32 Loss:  9270.16796875
Epoch:  33 Loss:  9269.39453125
Epoch:  34 Loss:  9269.310546875
Epoch:  35 Loss:  9268.197265625
Epoch:  36 Loss:  9268.271484375
Epoch:  37 Loss:  9268.107421875
Epoch:  38 Loss:  9267.693359375
Epoch:  39 Loss:  9266.77734375
Epoch:  40 Loss:  9266.7763671875
Total Precision, Recall, F score:  0.48694969215018025 0.5616577963990488 0.5216424525608858
F1 Score 0.5216424525608858
Epoch:  41 Loss:  9266.255859375
Epoch:  42 Loss:  9266.125
Epoch:  43 Loss:  9265.7158203125
Epoch:  44 Loss:  9265.625
Epoch:  45 Loss:  9265.4794921875
Epoch:  46 Loss:  9264.796875
Epoch:  47 Loss:  9264.912109375
Epoch:  48 Loss:  9264.6689453125
Epoch:  49 Loss:  9264.310546875
Epoch:  50 Loss:  9264.298828125
Total Precision, Recall, F score: 

In [0]:
# load = True

# model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(punc_dataset.word_to_ix), len(punc_dataset.tag_to_ix))
# if cuda:
#   model.cuda()
# loss_function = nn.NLLLoss()
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY) #, momentum=MOMENTUM)

# start_epoch = 1

# if load:
#   #LOAD_MODEL
#   model_name = "model_1_04192020_2322_80"
#   # Hard coding learning rate decay
#   # WEIGHT_DECAY = 0.99
#   model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(punc_dataset.word_to_ix), len(punc_dataset.tag_to_ix))
#   optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY) # momentum=MOMENTUM)
#   if cuda:
#     model.cuda()
#   path = "drive/My Drive/Colab Notebooks"
#   checkpoint = torch.load(path + "/models/" + CURR_NOTEBOOK + "/" + model_name)
#   model.load_state_dict(checkpoint['model_state_dict'])
#   optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#   start_epoch = checkpoint['epoch']
#   loss = checkpoint['loss']
#   print("Epoch from loaded model: ", start_epoch)
#   print("Loss from loaded model: ", loss.item())

In [15]:
get_test_accuracy(True)

{0: 184762, 1: 7117, 2: 8978, 3: 639}
{0: 5802, 1: 3474, 2: 6606, 3: 6622}
{0: 8337, 1: 6274, 2: 7482, 3: 411}
TEST Precision, Recall, F score of 1 :  0.671985648191861 0.5314763647225749 0.5935284796931033
TEST Precision, Recall, F score of 2 :  0.5761036960985626 0.5454434993924666 0.5603545125452503
TEST Precision, Recall, F score of 3 :  0.08800440710645917 0.6085714285714285 0.1537721092527975
TEST Total Precision, Recall, F score:  0.5004785261394904 0.5415358726254814 0.5201983306650918


0.5201983306650918